In [1]:
import pandas as pd
from app import WinrateCollect2

In [2]:
wc = WinrateCollect2("heroes_page_list.csv")
wc.update_stat()

In [67]:
# prepare heros stats
df = pd.read_pickle("data")
df['hero'] = df['hero'].str.replace('%20',' ')
df = df.astype({'g': 'int32'})
df.head()

,hero,g,wr,pos
0,Abaddon,61,50.0,3
1,Abaddon,74,55.6,4
2,Abaddon,1506,52.3,5
3,Alchemist,49,44.8,3
4,Ancient Apparition,99,29.3,4


In [68]:
# prepare heros matchups
df_m = pd.read_pickle("matchups")
df_m['hero'] = df_m['hero'].str.replace('%20',' ')
df_m = df_m.astype({'w': 'int32'})
df_m = df_m.astype({'l': 'int32'})
df_m.head()

,hero,matchup,w,l
0,Abaddon,Centaur Warrunner,15,1
1,Abaddon,Slardar,13,3
2,Abaddon,Zeus,17,6
3,Abaddon,Doom,50,24
4,Abaddon,Lich,12,4


In [80]:
def get_matchup(enemy_heroes):
    matchup = None
    for e in enemy_heroes:
        tmp = df_m.loc[(df_m['hero'] == e)][['matchup', 'w', 'l']].set_index('matchup') 
        if matchup is not None:
            matchup = matchup.add(tmp, fill_value=0)
        else:
            matchup = tmp
    matchup = matchup[~matchup.index.isin(enemy_heroes)]
    center_g = 1
    center_lr = 1
    matchup = matchup.assign(g = matchup['w'] + matchup['l'])
    matchup = matchup.assign(lr = matchup['l'] / (matchup['w'] + matchup['l']))
    matchup = matchup.assign(g_qant = matchup['g'].rank(method='max', pct=True))
    matchup = matchup.assign(lr_qant = matchup['lr'].rank(method='max', pct=True))
    matchup = matchup.assign(qant_circle = (matchup['g_qant']-center_g)**2 + (matchup['lr_qant']-center_lr)**2)
    matchup = matchup.assign(rank = 1 - matchup['qant_circle'])
    df_top = matchup.sort_values(by=['qant_circle'],ascending=True)[['g', 'lr']]
    df_top = df_top.loc[df_top['lr'] > 0.49]
    return df_top

In [82]:
df_tmp = df.sort_values(['hero', 'g'], ascending=[True,False]).drop_duplicates(['hero'])
p1 = list(df_tmp.loc[(df['pos'] == 1)].hero)
p2 = list(df_tmp.loc[(df['pos'] == 2)].hero)
p3 = list(df_tmp.loc[(df['pos'] == 3)].hero)
p4 = list(df_tmp.loc[(df['pos'] == 4)].hero)
p5 = list(df_tmp.loc[(df['pos'] == 5)].hero)

# insert enemy heroes
enemies = ['Ancient Apparition', 'Hoodwink']
m = get_matchup(enemies)

# select pos
m[m.index.isin(p5)]

,g,lr
matchup,,
Abaddon,465.0,0.541935
Winter Wyvern,575.0,0.525217
Keeper of the Light,251.0,0.541833
Phoenix,208.0,0.533654
Treant Protector,158.0,0.544304
Disruptor,196.0,0.515306
Enchantress,165.0,0.496970
Bane,99.0,0.555556
Io,108.0,0.527778


In [15]:
df_tmp = df.sort_values(['hero', 'g'], ascending=[True,False]).drop_duplicates(['hero'])
df_tmp

,hero,g,wr,pos
2,Abaddon,1506,52.3,5
3,Alchemist,49,44.8,3
5,Ancient Apparition,2092,51.5,5
6,Anti-Mage,764,46.0,1
7,Arc Warden,155,45.5,1
...,...,...,...,...
268,Windranger,728,50.1,4
273,Winter Wyvern,1697,55.2,5
274,Witch Doctor,435,47.4,5
275,Wraith King,612,49.0,1


In [24]:
p1 = list(df_tmp.loc[(df['pos'] == 1)].hero)

In [6]:
pos = 1
# & (df['wr'] >= 52)
min_g = df.loc[(df['pos'] == pos)]['g'].max() / 100
df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) ]
df_top3 = df1.sort_values(by=['wr'],ascending=False)
df_top3

,hero,g,wr,pos
201,Shadow Fiend,45,78.6,1
186,Razor,130,59.6,1
18,Bounty Hunter,33,58.3,1
210,Slark,495,58.0,1
121,Luna,2563,55.1,1
24,Bristleback,104,54.8,1
87,Gyrocopter,1500,54.7,1
133,Medusa,1084,54.6,1
80,Faceless Void,1164,54.1,1
115,Lifestealer,388,53.3,1


In [8]:
'''
# old version
def tierS(pos):
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 3
    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) & (df['wr'] >= 52)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3

def tierA(pos):
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 4
    df2 = tierS(pos)
    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g)  & (df['wr'] >= 49)]
    df1 = df1.sort_values(by=['wr'],ascending=False).head(14)
    df_top8 = pd.concat([df1, df2]).drop_duplicates(keep=False)
    return df_top8.head(11)

def topWR():
    min_g = df['g'].max() / 40
    df_top3 = df.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)

def topTier():
    min_g = df['g'].max() / 10
    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3.head(15)
'''

"\n# old version\ndef tierS(pos):\n    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 3\n    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g) & (df['wr'] >= 52)]\n    df_top3 = df1.sort_values(by=['wr'],ascending=False)\n    return df_top3\n\ndef tierA(pos):\n    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 4\n    df2 = tierS(pos)\n    df1 = df.loc[(df['pos'] == pos) & (df['g'] > min_g)  & (df['wr'] >= 49)]\n    df1 = df1.sort_values(by=['wr'],ascending=False).head(14)\n    df_top8 = pd.concat([df1, df2]).drop_duplicates(keep=False)\n    return df_top8.head(11)\n\ndef topWR():\n    min_g = df['g'].max() / 40\n    df_top3 = df.sort_values(by=['wr'],ascending=False)\n    return df_top3.head(10)\n\ndef topTier():\n    min_g = df['g'].max() / 10\n    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]\n    df_top3 = df1.sort_values(by=['wr'],ascending=False)\n    return df_top3.head(15)\n"

In [19]:
# make games count less valuable
#center_g = 0.9

# make win rate less valuable
#center_wr = 0.9

center_g = 1
center_wr = 1


#.style.hide_index()
def tierS(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[df1['qant_circle'] <= 0.0625]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top

def tierA(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[(df1['qant_circle'] <= 0.25) & (df1['qant_circle'] > 0.0625)]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top

def tierB_plus(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[(df1['qant_circle'] > 0.25)]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top

def topWR_lowpick():
    df_top3 = df.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


def topHiddenImba_pos(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g/2)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[df1['qant_circle'] <= 0.25]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    return df_top.head(3)
    
def topWinrateOverallTiers():
    min_g = df['g'].max() / 10
    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


In [20]:
topWinrateOverallTiers()

,hero,g,wr,pos
272,Winter Wyvern,380,64.8,4
99,Io,332,63.2,5
12,Batrider,612,60.1,2
28,Broodmother,1102,59.6,3
243,Undying,315,59.1,5
156,Nyx Assassin,1354,58.4,4
210,Slark,495,58.0,1
41,Dark Willow,963,56.8,4
158,Ogre Magi,377,55.6,5
273,Winter Wyvern,1641,55.2,5


In [21]:
topWR_lowpick()

,hero,g,wr,pos
40,Dark Willow,19,87.5,3
201,Shadow Fiend,45,78.6,1
109,Legion Commander,25,77.8,5
147,Naga Siren,19,75.0,4
98,Io,17,75.0,4
169,Pangolier,33,75.0,5
100,Jakiro,28,75.0,3
259,Void Spirit,72,67.7,3
236,Tiny,21,66.7,5
184,Pugna,85,66.7,5


In [22]:
tierS(1)

,hero,g,wr,rank
121,Luna,2563,55.1,0.994898
87,Gyrocopter,1500,54.7,0.983418
80,Faceless Void,1164,54.1,0.947704
133,Medusa,1084,54.6,0.947704


In [23]:
tierA(1)

,hero,g,wr,rank
171,Phantom Lancer,1446,50.8,0.834184
244,Ursa,831,50.6,0.753827


In [24]:
tierB_plus(1)

,hero,g,wr,rank
144,Morphling,880,50.3,0.738520
210,Slark,495,58.0,0.711735
222,Templar Assassin,501,52.9,0.668367
115,Lifestealer,388,53.3,0.585459
275,Wraith King,643,49.0,0.519133
219,Sven,526,49.4,0.497449
6,Anti-Mage,726,46.0,0.436224
140,Monkey King,340,51.6,0.436224
189,Riki,270,53.3,0.391582
224,Terrorblade,1759,44.3,0.264031


In [25]:
tierS(2)

,hero,g,wr,rank
72,Ember Spirit,1207,54.9,0.975586
223,Templar Assassin,1388,54.0,0.960938


In [26]:
tierA(2)

,hero,g,wr,rank
12,Batrider,612,60.1,0.919922
116,Lina,1168,52.3,0.905273
175,Puck,2167,51.2,0.859375
166,Pangolier,434,55.0,0.771484


In [27]:
tierS(3)

,hero,g,wr,rank
28,Broodmother,1102,59.6,0.978374


In [28]:
tierA(3)

,hero,g,wr,rank
108,Legion Commander,663,54.6,0.936851
167,Pangolier,800,52.2,0.915225
39,Dark Seer,994,51.6,0.898789
130,Mars,1761,50.8,0.895329
44,Dawnbreaker,450,53.9,0.861592
198,Sand King,1257,50.5,0.846021
252,Viper,471,51.2,0.808824
15,Beastmaster,386,51.8,0.775087


In [29]:
tierS(4)

,hero,g,wr,rank
156,Nyx Assassin,1354,58.4,0.992188
41,Dark Willow,963,56.8,0.966797
138,Mirana,1819,53.2,0.952148


In [30]:
tierA(4)

,hero,g,wr,rank
119,Lion,1262,52.9,0.928711
179,Pudge,630,51.5,0.823242


In [31]:
topHiddenImba_pos(4)

,hero,g,wr,rank
272,Winter Wyvern,380,64.8,0.999023
203,Shadow Shaman,359,54.3,0.971680
45,Dawnbreaker,299,54.2,0.902344


In [32]:
tierS(5)

,hero,g,wr,rank
273,Winter Wyvern,1641,55.2,0.992695


In [33]:
tierA(5)

,hero,g,wr,rank
94,Hoodwink,767,53.1,0.934989
174,Phoenix,517,53.9,0.934989
2,Abaddon,1271,52.3,0.924032
5,Ancient Apparition,1986,51.5,0.894814
157,Nyx Assassin,429,53.0,0.852447
158,Ogre Magi,377,55.6,0.810080
238,Treant Protector,501,50.5,0.776479
120,Lion,820,49.9,0.751644


In [34]:
topHiddenImba_pos(5)

,hero,g,wr,rank
99,Io,332,63.2,0.999817
243,Undying,315,59.1,0.997626
158,Ogre Magi,377,55.6,0.992513


In [40]:
import os
from datetime import datetime
fold = 'history/'+ datetime.today().strftime('%Y_%m_%d__%Hh_%Mm')
os.mkdir(fold)

In [41]:
import dataframe_image as dfi

In [51]:
tierS(1).dfi.export(fold + '/1_TierS.png')
tierA(1).dfi.export(fold + '/1_TierA.png')
tierB_plus(1).dfi.export(fold + '/1_TierBplus.png')
topWinrateOverallTiers().dfi.export(fold + '/topWinrate.png')
topWR_lowpick().dfi.export(fold + '/topLowpick.png')
tierS(2).dfi.export(fold + '/2_TierS.png')
tierA(2).dfi.export(fold + '/2_TierA.png')
tierS(3).dfi.export(fold + '/3_TierS.png')
tierA(3).dfi.export(fold + '/3_TierA.png')
tierS(4).dfi.export(fold + '/4_TierS.png')
tierA(4).dfi.export(fold + '/4_TierA.png')
topHiddenImba_pos(4).dfi.export(fold + '/4_HiddenImba.png')
tierS(5).dfi.export(fold + '/5_TierS.png')
tierA(5).dfi.export(fold + '/5_TierA.png')
topHiddenImba_pos(5).dfi.export(fold + '/5_HiddenImba.png')

In [55]:
from IPython.display import display, HTML

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [61]:
display(tierS(1))
display(tierA(1))
#display(tierB_plus(1))

,hero,g,wr,rank
121,Luna,2563,55.1,0.994898
87,Gyrocopter,1500,54.7,0.983418
80,Faceless Void,1164,54.1,0.947704
133,Medusa,1084,54.6,0.947704


,hero,g,wr,rank
171,Phantom Lancer,1446,50.8,0.834184
244,Ursa,831,50.6,0.753827


In [7]:
df1 = df.loc[(df['pos'] == 1)]
df1 = df1.sort_values(by=['g'],ascending=False)
df1

,hero,g,wr,pos
117,Luna,2542,55.6,1
213,Terrorblade,1840,48.2,1
164,Phantom%20Lancer,1422,51.3,1
84,Gyrocopter,1238,55.3,1
77,Faceless%20Void,1114,52.7,1
204,Spectre,956,45.7,1
130,Medusa,952,53.1,1
232,Ursa,856,50.0,1
137,Morphling,852,45.9,1
264,Wraith%20King,704,48.8,1


In [32]:
df1.describe()

,g,wr,g_qant,wr_qant
count,40.000000,40.000000,40.000000,40.000000
mean,437.750000,48.987500,0.513125,0.514375
std,408.129991,6.155306,0.291492,0.291486
min,104.000000,34.700000,0.025000,0.025000
25%,164.750000,44.275000,0.268750,0.268750
50%,325.000000,49.550000,0.512500,0.512500
75%,479.000000,51.675000,0.756250,0.756250
max,1964.000000,66.700000,1.000000,1.000000


In [52]:
# make games count less valuable
#center_g = 0.9

# make win rate less valuable
#center_wr = 0.9

center_g = 1
center_wr = 1


#.style.hide_index()
def tiers(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df1['tier'] = ''
    df1['tier'] = df1['qant_circle'].apply(lambda x: 'S' if x <= 0.0625 else 
                                           ('A' if (x <= 0.25) & (x > 0.0625) else 
                                           ('B' if (x <= 0.5625) & (x > 0.25) else 'C+')))
    
    df_top = df1.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank', 'tier']]
    df_top.index = range(1, len(df1.index) + 1)
    return df_top


def topWR_lowpick():
    df_top3 = df.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


def topHiddenImba_pos(pos):
    # 5% - outliers
    min_g = df.loc[(df['pos'] == pos)]['g'].max() / 20
    df1 = df.loc[(df['pos'] == pos) & (df['g'] >= min_g)]
    
    df1 = df1.assign(g_qant = df1['g'].rank(method='max', pct=True))
    df1 = df1.assign(wr_qant = df1['wr'].rank(method='max', pct=True))
    df1 = df1.assign(qant_circle = (df1['g_qant']-center_g/2)**2 + (df1['wr_qant']-center_wr)**2)
    df1 = df1.assign(rank = 1 - df1['qant_circle'])
    
    df2 = df1.loc[df1['qant_circle'] <= 0.25]
    df_top = df2.sort_values(by=['qant_circle'],ascending=True)[['hero', 'g', 'wr', 'rank']]
    df_top.index = range(1, len(df2.index) + 1)
    return df_top.head(10)
    
def topWinrateOverallTiers():
    min_g = df['g'].max() / 10
    df1 = df.loc[(df['g'] > min_g)  & (df['wr'] >= 49)]
    df_top3 = df1.sort_values(by=['wr'],ascending=False)
    return df_top3.head(10)


In [53]:
from IPython.display import display, HTML

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [54]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
subset = ['rank']

a = tiers(5)
a = a.style.background_gradient('RdYlGn', subset = subset)
a

,hero,g,wr,rank,tier
1,Winter Wyvern,1641,55.200000,0.992695,S
2,Phoenix,517,53.900000,0.934989,A
3,Hoodwink,767,53.100000,0.934989,A
4,Abaddon,1271,52.300000,0.924032,A
5,Ancient Apparition,1986,51.500000,0.894814,A
6,Nyx Assassin,429,53.000000,0.852447,A
7,Ogre Magi,377,55.600000,0.810080,A
8,Treant Protector,501,50.500000,0.776479,A
9,Lion,820,49.900000,0.751644,A
10,Io,332,63.200000,0.736304,B


In [40]:
display(tiers(4)), 
display(topHiddenImba_pos(4))

,hero,g,wr,rank,tier
1,Nyx Assassin,1354,58.4,0.992188,S
2,Dark Willow,963,56.8,0.966797,S
3,Mirana,1819,53.2,0.952148,S
4,Lion,1262,52.9,0.928711,A
5,Pudge,630,51.5,0.823242,A
6,Winter Wyvern,380,64.8,0.717773,B
7,Skywrath Mage,545,50.9,0.716797,B
8,Windranger,717,50.1,0.702148,B
9,Shadow Shaman,359,54.3,0.659180,B
10,Earth Spirit,869,47.7,0.612305,B


,hero,g,wr,rank
1,Winter Wyvern,380,64.8,0.999023
2,Shadow Shaman,359,54.3,0.971680
3,Dawnbreaker,299,54.2,0.902344
4,Dark Willow,963,56.8,0.873047
5,Pudge,630,51.5,0.854492
6,Riki,175,55.1,0.843750
7,Phoenix,329,51.3,0.811523
8,Skywrath Mage,545,50.9,0.810547
9,Bounty Hunter,386,50.7,0.808594
10,Invoker,98,63.4,0.807617
